# Airbnb MCP + AWS Bedrock Integration Lab

This lab demonstrates how to use **LiteLLM** as a unified interface to access **AWS Bedrock** models (specifically Claude 3.5 Sonnet) with **Airbnb MCP server** for property search and booking research.

## Key Benefits:
- 🔄 **Unified API**: LiteLLM provides OpenAI-compatible interface for 100+ models
- 💰 **Cost Effective**: AWS Bedrock often 60-70% cheaper than OpenAI
- 🔒 **Privacy**: Data stays in your AWS account
- 🏠 **Airbnb Integration**: Search properties and get detailed listing information
- 🌍 **Regional**: Using EU region models

## Setup Required:
1. **AWS credentials** with Bedrock access
2. **Claude 3.5 Sonnet** model access in AWS Bedrock
3. **Node.js** and npm/npx for MCP servers
4. **Airbnb MCP server** - Clone and build:
   ```bash
   git clone https://github.com/openbnb-org/mcp-server-airbnb.git
   cd mcp-server-airbnb
   npm install
   npm run build
   ```

## Prerequisites Check:
- Verify Node.js is installed: `node --version`
- Verify npm is available: `npm --version`
- Verify npx is available: `npx --version`

If Node.js is not installed, please install it from [nodejs.org](https://nodejs.org/) before proceeding.

## Important Note - Robots.txt:
This lab uses the `--ignore-robots-txt` flag to bypass Airbnb's robots.txt restrictions for **educational testing purposes only**. The MCP server respects robots.txt by default as an ethical web scraping practice. 

⚠️ **Please use responsibly and respect website terms of service and rate limits.**

## Import Required Libraries

In [ ]:
# Import required libraries and AWS utilities
import asyncio
import os
from agents import Agent, Runner, trace, set_tracing_disabled
from agents.mcp import MCPServerStdio

# Import AWS utilities from common module
import sys
sys.path.append('..')
from aws_utils import setup_aws_credentials, test_bedrock_connectivity

# Disable OpenAI tracing since we're using Bedrock
set_tracing_disabled(True)
print("🚫 OpenAI tracing disabled (using Bedrock instead)")

In [ ]:
# Set up AWS credentials and test Bedrock connectivity
aws_access_key, aws_secret_key, aws_session_token, aws_region = setup_aws_credentials('eu-west-1')

# Model configuration
model_id = "eu.anthropic.claude-3-5-sonnet-20240620-v1:0"
print(f"\n🤖 Using model: {model_id}")

# Test Bedrock connectivity if credentials were retrieved
if aws_access_key:
    test_bedrock_connectivity(aws_access_key, aws_secret_key, aws_session_token, aws_region, model_id)
else:
    print("⚠️ Skipping Bedrock connectivity test due to missing credentials")

In [ ]:
# Set up MCP server configurations

# 🚨 IMPORTANT: Update the path below to point to your local mcp-server-airbnb directory
# 
# Step 1: Clone the Airbnb MCP server repository:
#   git clone https://github.com/openbnb-org/mcp-server-airbnb.git
# 
# Step 2: Build the server:
#   cd mcp-server-airbnb && npm install && npm run build
#
# Step 3: Update the path below to match your local setup:

airbnb_server_path = "/Users/yongqiwu/code/mcp-server-airbnb"  # ⚠️ UPDATE THIS PATH!

# Configure the MCP server parameters - using built version for reliability
# Adding --ignore-robots-txt flag to bypass Airbnb's robots.txt restrictions for educational testing
# NOTE: The server respects robots.txt by default as an ethical web scraping practice
airbnb_params = {"command": "node", "args": [f"{airbnb_server_path}/dist/index.js", "--ignore-robots-txt"]}

# Alternative: Use TypeScript source (requires tsx to be installed globally)
# airbnb_params = {"command": "npx", "args": ["-p", airbnb_server_path, "tsx", f"{airbnb_server_path}/index.ts", "--ignore-robots-txt"]}

print(f"🏠 Airbnb MCP server path: {airbnb_server_path}")
print("🔧 MCP server configurations ready")
print("⚠️  Note: Using --ignore-robots-txt for educational testing only")

In [ ]:
# Test Airbnb MCP Server
print("🏠 Testing Airbnb MCP Server...")
try:
    async with MCPServerStdio(params=airbnb_params, client_session_timeout_seconds=60) as server:
        airbnb_tools = await server.list_tools()
    
    print("✅ Airbnb MCP Server connected successfully!")
    print("🛠️ Airbnb tools available:")
    for tool in airbnb_tools:
        print(f"  - {tool.name}: {tool.description}")
        
except Exception as e:
    print(f"❌ Failed to connect to Airbnb MCP server: {e}")
    print("   Make sure the server is built and dependencies are installed")
    print(f"   Try running: cd {airbnb_server_path} && npm install && npm run build")

## Run the Agent Task

Let's create an agent that can search for Airbnb properties using **Claude 3.5 Sonnet via AWS Bedrock**:

In [ ]:
# Main agent execution
async def run_airbnb_agent(query):
    """Run the Airbnb agent with proper MCP server context management."""
    
    # Start Airbnb MCP server
    async with MCPServerStdio(params=airbnb_params, client_session_timeout_seconds=60) as mcp_server_airbnb:
        print("✅ Airbnb MCP server connected")
        
        # Create agent with Airbnb server
        agent = Agent(
            name="airbnb_assistant", 
            instructions=instructions, 
            model=f"litellm/bedrock/{model_id}",
            mcp_servers=[mcp_server_airbnb]
        )
        
        print("🤖 Agent created with Claude 3.5 Sonnet via AWS Bedrock")
        print(f"📝 Task: {query}")
        
        # Run the task within the MCP server context
        result = await Runner.run(agent, query)
        
        print("\n" + "="*50)
        print("✅ TASK COMPLETED!")
        print("="*50)
        print(result.final_output)
        
        return result

# Example query
sample_query = "Find me a nice Airbnb in Paris for 2 adults from September 15-20, 2025. Budget around $100-200 per night."
print(f"🔍 Sample query: {sample_query}")

In [20]:
# Run the agent with the sample query
result = await run_airbnb_agent(sample_query)

✅ Airbnb MCP server connected
🔧 Airbnb tools available: ['airbnb_search', 'airbnb_listing_details']
✅ Filesystem MCP server connected
🔍 Checking agent tools...
⚠️ Could not check agent tools: RunContextWrapper.__init__() missing 1 required positional argument: 'context'
🤖 Agent created with Claude 3.5 Sonnet via AWS Bedrock
📝 Task: Find me a nice Airbnb in Paris for 2 adults from September 15-20, 2025. Budget around $100-200 per night.

✅ TASK COMPLETED!
Based on the search results, I've found several nice Airbnb options in Paris that fit your criteria for 2 adults from September 15-20, 2025, with a budget of $100-200 per night. Here are some of the best options:

1. Studio with terrace near the center of Paris
   - Price: $706 SGD (approximately $526 USD) for 5 nights
   - Rating: 4.85/5 (269 reviews)
   - Features: 1 queen bed, terrace
   - Link: https://www.airbnb.com/rooms/30485598

2. Charming Parisian studio
   - Price: $993 SGD (approximately $740 USD) for 5 nights
   - Rating: 